<a href="https://colab.research.google.com/github/danielveigasilva/SMNLMS-SR/blob/main/SMNLMS_SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysr

import numpy as np
import pandas as pd
from pysr import PySRRegressor
import sys

def evaluateEmpiricalQuantitiesSMNLMS(tauVector , betaVector , NVector , sigmanu2Vector , sigmax2Vector , numberOfIterations , numberOfLastIterations , numberOfRepeats):

	MSE = np.zeros((len(tauVector),len(betaVector),len(NVector),len(sigmanu2Vector),len(sigmax2Vector)))
	MSD = np.zeros((len(tauVector),len(betaVector),len(NVector),len(sigmanu2Vector),len(sigmax2Vector)))
	Pup = np.zeros((len(tauVector),len(betaVector),len(NVector),len(sigmanu2Vector),len(sigmax2Vector)))

	for tauIndex in range(len(tauVector)):
		for betaIndex in range(len(betaVector)):
			for NIndex in range(len(NVector)):
				for sigmanu2Index in range(len(sigmanu2Vector)):
					for sigmax2Index in range(len(sigmax2Vector)):
						print(str(tauIndex + 1) + '/' + str(len(tauVector)) + ' - ' + str(betaIndex + 1) + '/' + str(len(betaVector)) + ' - ' + str(NIndex + 1) + '/' + str(len(NVector)) + ' - ' + str(sigmanu2Index + 1) + '/' + str(len(sigmanu2Vector)) + ' - ' + str(sigmax2Index + 1) +  '/' + str(len(sigmax2Vector)))
						tau = tauVector[tauIndex]
						beta = betaVector[betaIndex]
						N = NVector[NIndex]
						sigmanu2 = sigmanu2Vector[sigmanu2Index]
						sigmax2 = sigmax2Vector[sigmax2Index]

						for repeat in range(numberOfRepeats):
							wk = np.zeros((N,1))
							w0 = np.random.randn(N,1)
							x  = np.sqrt(sigmax2) * np.random.randn( numberOfIterations + N - 1, 1 )
							d  = np.convolve(w0[:,0], x[:,0])
							d  += np.sqrt( sigmanu2 ) * np.random.randn(len(d))
							gamma = np.sqrt(tau * sigmanu2)

							for k in range(N, numberOfIterations + N - 1):
								iteration = k - N + 1
								xk = x[k:k-N:-1]
								yk = np.dot(wk.T, xk)
								ek = d[k] - yk

								if abs(ek) > gamma:
									mu = 1 - gamma/abs(ek)
									wk = wk + beta * mu / (np.dot(xk.T, xk)) * ek * xk

									if iteration > numberOfIterations - numberOfLastIterations + 1:
										Pup[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index] += 1 / (numberOfRepeats * numberOfLastIterations)

								if iteration > numberOfIterations - numberOfLastIterations + 1:
									MSD[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index] += np.linalg.norm(wk - w0) ** 2 / (numberOfRepeats * numberOfLastIterations)
									MSE[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index] += ek.item() ** 2 / (numberOfRepeats * numberOfLastIterations)
	return MSE,MSD,Pup


def RegressaoSimbolica(X,y):
	model = PySRRegressor(
		parsimony = 0.0001, # padrão = 0.0032
		progress=False,
		niterations=100,
		weight_randomize=0.001, # default: 0.00023
		populations=300,
		population_size=100,
		model_selection='score', # 'accuracy', 'best', or 'score'
		nested_constraints={"exp":{"exp": 0},"log10":{"log10": 0},"erf":{"erf": 0},"erfc":{"erfc": 0}},
		binary_operators=["+", "*","-","^"],
		unary_operators=[
			"exp",
			"inv(x) = 1/x",
			"log10",
			"erf",
			"erfc",
		],
		extra_sympy_mappings={"inv": lambda x: 1 / x})

	model.fit(X, y)
	print(model)

	best_idx = model.equations_.query(
		f"loss < {2 * model.equations_.loss.min()}"
	).score.idxmax()
	model.sympy(best_idx)

	model.get_best().equation



tauVector = np.arange(0, 6)
betaVector = np.arange(0.1, 1.1, 0.1)
NVector = [10, 20, 50]
sigmanu2Vector = np.power(10,np.arange(-6, -1), dtype=float)
sigmax2Vector = np.power(10,np.arange(-2, 2), dtype=float)
numberOfIterations = 50000
numberOfLastIterations = 1000
numberOfRepeats = 100

tauVector = np.arange(0, 2)
betaVector = np.arange(0.1, 1.1, 0.1)
NVector = [10]
sigmanu2Vector = np.power(10,np.arange(-6, -1), dtype=float)
sigmax2Vector = np.power(10,np.arange(1, 2), dtype=float)
numberOfIterations = 50
numberOfLastIterations = 1000
numberOfRepeats = 100

print('>> Geradando dados SMNLMS...')
MSE,MSD,Pup = evaluateEmpiricalQuantitiesSMNLMS(tauVector,betaVector,NVector,sigmanu2Vector,sigmax2Vector,numberOfIterations,numberOfLastIterations,numberOfRepeats)

data = {
    'tau': [],
    'beta': [],
    'N': [],
    'sigmanu2': [],
    'sigmax2': [],
	'MSE': [],
    'MSD': [],
    'Pup': [],
}


for tauIndex in range(len(tauVector)):
		for betaIndex in range(len(betaVector)):
			for NIndex in range(len(NVector)):
				for sigmanu2Index in range(len(sigmanu2Vector)):
					for sigmax2Index in range(len(sigmax2Vector)):
						data['tau'].append(tauVector[tauIndex])
						data['beta'].append(betaVector[betaIndex])
						data['N'].append(NVector[NIndex])
						data['sigmanu2'].append(sigmanu2Vector[sigmanu2Index])
						data['sigmax2'].append(sigmax2Vector[sigmax2Index])
						data['MSE'].append(MSE[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index])
						data['MSD'].append(MSD[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index])
						data['Pup'].append(Pup[tauIndex , betaIndex , NIndex , sigmanu2Index , sigmax2Index])

file = './resultsSMNLMS.csv'

print('>> Gerando arquivo csv gerado SMNLMS')
df = pd.DataFrame(data)
df.to_csv(file, index=False, header=False)

print('>> Iniciando Regração Simbólica...')

arr = np.loadtxt( file , delimiter = ',' , dtype = float)

#tau beta N sigmanu2 sigmax2
X = arr[ : ,  [ 0 , 1 , 2 , 3 , 4 ] ]
#/usr/local/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/Python
#/usr/local/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/Python

print('>> MSE')
RegressaoSimbolica(X, arr[ : , 5 ])
print('>> MSD')
RegressaoSimbolica(X, arr[ : , 6 ])
print('>> Pup')
RegressaoSimbolica(X, arr[ : , 7 ])

print('>> FIM!')

>> Geradando dados SMNLMS...
1/2 - 1/10 - 1/1 - 1/5 - 1/1
1/2 - 1/10 - 1/1 - 2/5 - 1/1
1/2 - 1/10 - 1/1 - 3/5 - 1/1
1/2 - 1/10 - 1/1 - 4/5 - 1/1
1/2 - 1/10 - 1/1 - 5/5 - 1/1
1/2 - 2/10 - 1/1 - 1/5 - 1/1
1/2 - 2/10 - 1/1 - 2/5 - 1/1
1/2 - 2/10 - 1/1 - 3/5 - 1/1
1/2 - 2/10 - 1/1 - 4/5 - 1/1
1/2 - 2/10 - 1/1 - 5/5 - 1/1
1/2 - 3/10 - 1/1 - 1/5 - 1/1
1/2 - 3/10 - 1/1 - 2/5 - 1/1
1/2 - 3/10 - 1/1 - 3/5 - 1/1
1/2 - 3/10 - 1/1 - 4/5 - 1/1
1/2 - 3/10 - 1/1 - 5/5 - 1/1
1/2 - 4/10 - 1/1 - 1/5 - 1/1
1/2 - 4/10 - 1/1 - 2/5 - 1/1
1/2 - 4/10 - 1/1 - 3/5 - 1/1
1/2 - 4/10 - 1/1 - 4/5 - 1/1
1/2 - 4/10 - 1/1 - 5/5 - 1/1
1/2 - 5/10 - 1/1 - 1/5 - 1/1
1/2 - 5/10 - 1/1 - 2/5 - 1/1
1/2 - 5/10 - 1/1 - 3/5 - 1/1
1/2 - 5/10 - 1/1 - 4/5 - 1/1
1/2 - 5/10 - 1/1 - 5/5 - 1/1
1/2 - 6/10 - 1/1 - 1/5 - 1/1
1/2 - 6/10 - 1/1 - 2/5 - 1/1
1/2 - 6/10 - 1/1 - 3/5 - 1/1
1/2 - 6/10 - 1/1 - 4/5 - 1/1
1/2 - 6/10 - 1/1 - 5/5 - 1/1
1/2 - 7/10 - 1/1 - 1/5 - 1/1
1/2 - 7/10 - 1/1 - 2/5 - 1/1
1/2 - 7/10 - 1/1 - 3/5 - 1/1
1/2 - 7/10 - 1

/usr/local/lib/python3.10/dist-packages/pysr/sr.py:65: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.410e+03
Head worker occupation: 1.0%
Progress: 3 / 30000 total iterations (0.010%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           7.486e-01  7.971e+00  y = erfc(-0.050372)
3           1.271e-01  1.773e+00  y = exp(erfc(x₁))
5           1.234e-01  1.476e-02  y = exp(erfc(x₁ * 1.5593))
6           1.883e-02  1.880e+00  y = erf(x₁) ^ log10(erf(0.26367))
7           1.846e-02  2.004e-02  y = (x₁ ^ log10(erf(0.26367))) - 0.006261
8           1.809e-02  2.037e-02  y = (erf(x₁) ^ log10(erf(0.26367))) - 0.006261
9           1.197e-02  4.130e-01  y = (x₁ ^ log10(erf(0.26367))) - (0.006261 * x₄)
10          1.156e-02  3.472e-02  y = (x₁ ^ log10(erf(0.26367))) - (erf(0.006261) * x₄)
11          1.052e-02  9.378e-02  y = (x₁ ^ erf(-0.51558)) - (0.0068029 * (inv(x₁) + x₄))
12          1.036e-02  1.562e-02  y = (x₁ ^ log10(erf(0.22488))) - (0.0